d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

# 2.4 Shuffle Partitions

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this notebook you:<br>
* Understand the performance differences between wide and narrow transformations.
* Optimize Spark jobs by configuring Shuffle Partitions.

Create the table if it doesn't exist.

In [0]:
%run ../Includes/Classroom-Setup

Data mounted to /mnt/davis ...

Let's see the most common call types in our dataset. Any guesses?

In [0]:
%sql
SELECT `call type`, count(*) AS count
FROM firecalls
GROUP BY `call type`
ORDER BY count DESC

call type,count
Medical Incident,156374
Structure Fire,31329
Alarms,26090
Traffic Collision,9749
Other,3799
Citizen Assist / Service Call,3600
Outside Fire,2940
Vehicle Fire,1101
Water Rescue,1096
Gas Leak (Natural and LP Gases),888


-sandbox
## What is that 200/200?

Expand out the Spark job above. It should have:
* 1 stage with 2 tasks
* 1 stage with 200 tasks

The number assigned to the Job/Stage will depend on how many Spark jobs you have already executed on your cluster.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> The Community Edition version of Databricks now provisions a single driver node with 2 cores for you.  This change explains why your output might look different from the video.

-sandbox
**Narrow Transformations**: The data required to compute the records in a single partition reside in at most one partition of the parent DataFrame.

Examples include:
* `SELECT (columns)`
* `DROP (columns)`
* `WHERE`

<img src="https://files.training.databricks.com/images/eLearning/ucdavis/transformations-narrow.png" alt="Narrow Transformations" style="height:300px"/>

<br/>

**Wide Transformations**: The data required to compute the records in a single partition may reside in many partitions of the parent DataFrame. 

Examples include:
* `DISTINCT` 
* `GROUP BY` 
* `ORDER BY` 

<img src="https://files.training.databricks.com/images/eLearning/ucdavis/transformations-wide.png" alt="Wide Transformations" style="height:300px"/>

-sandbox
## Shuffle Partitions

The `spark.sql.shuffle.partitions` parameter controls how many resulting partitions there are after a shuffle (wide transformation). By default, this value is 200 regardless of how large or small your dataset is, or your cluster configuration.

Let's change this parameter to be 8 (default parallelism in Databricks Community edition is 2 because that is the number of threads we have available).

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> This configuration will only be changed for this notebook (it will default to 200 if you switch notebooks/detach from your cluster). If you want to set this parameter for all of your clusters, you can also set this configuration at time of cluster creation.

<div><br><img src="https://files.training.databricks.com/images/eLearning/ucdavis/sqlConf.png" style="height: 300px; border: 1px solid #aaa; box-shadow: 5px 5px 5px #aaa; margin: 20px"/></div>

In [0]:
%sql
SET spark.sql.shuffle.partitions=8

key,value
spark.sql.shuffle.partitions,8


Let's try this again...

In [0]:
%sql
SELECT `call type`, count(*) AS count
FROM firecalls
GROUP BY `call type`
ORDER BY count DESC

call type,count
Medical Incident,156374
Structure Fire,31329
Alarms,26090
Traffic Collision,9749
Other,3799
Citizen Assist / Service Call,3600
Outside Fire,2940
Vehicle Fire,1101
Water Rescue,1096
Gas Leak (Natural and LP Gases),888


Wow! That was a bit faster, and we didn't have to change any of our SQL query code!

## Extension

Try changing the shuffle partitions parameter to different values (e.g. 8, 64, 100, 400) and see how it impacts the performance.

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>